In [1]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

In [2]:
const EID_CREDENTIAL = {
    name: "Michael Jackson",
    age: "50",
    profession: "RSE",
    project: "EID"
}
const CREDENTIAL_DATA = Object.values(EID_CREDENTIAL)

In [3]:
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

In [4]:
import { BBSPlusSignatureParamsG1, BBSPlusKeypairG2 } from '@docknetwork/crypto-wasm-ts';
const messageCount = CREDENTIAL_DATA.length;

const label = stringToBytes("My signature params");

// Deterministically generated params
const paramsDeterministc = BBSPlusSignatureParamsG1.generate(messageCount, label);

// Generating a keypair once signature parameters are created.

const keypair = BBSPlusKeypairG2.generate(paramsDeterministc);
const sk = keypair.secretKey;
const pk = keypair.publicKey;


In [5]:
import { BBSPlusSignatureG1 } from '@docknetwork/crypto-wasm-ts'
// Encoding messages
const encodedMessages = CREDENTIAL_DATA.map(element => stringToBytes(element));

// The signing function will encode bytes to a field element as true is passed
const sig = BBSPlusSignatureG1.generate(encodedMessages, sk, paramsDeterministc, true);

// As the messages are not encoded, pass true to the verification function to make it encode messages before verifying the signature.
sig.verify(encodedMessages, pk, paramsDeterministc, true);

{ verified: true, error: undefined }


# Selective Disclosure 

In [6]:
import {Statement, Statements, MetaStatements, ProofSpec, Witness, Witnesses, CompositeProof} from '@docknetwork/crypto-wasm-ts';

// Prover prepares the attributes he wants to disclose, i.e. attribute index 2 and 4 (indexing is 0-based), and the ones he wants to hide. 
const revealedMsgIndices: Set<number> = new Set([2, 1]); 

// revealedMsgs are the attributes disclosed to the verifier
const revealedMsgs: Map<number, Uint8Array> = new Map([
  [1, encodedMessages[1]],
  [2, encodedMessages[2]]
]); 

const unrevealedMsgs: Map<number, Uint8Array> = new Map([
  [0, encodedMessages[0]],
  [3, encodedMessages[3]]
]); 

// Create a BBS signature, true indicates that attributes/messages are arbitrary bytes and should be encoded first
const statement1 = Statement.bbsPlusSignatureProverConstantTime(paramsDeterministc, revealedMsgs, true);
const statements = new Statements();
statements.add(statement1);

const ms = new MetaStatements();
const proofSpec = new ProofSpec(statements, ms, [], stringToBytes("Awesome Proof - test 001"));

const witness1 = Witness.bbsPlusSignatureConstantTime(sig, unrevealedMsgs, true);
const witnesses = new Witnesses();
witnesses.add(witness1);

const nonce = stringToBytes('a unique nonce given by verifier');
const proof = CompositeProof.generate(proofSpec, witnesses, nonce);



In [7]:
const verifierStatement1 = Statement.bbsPlusSignatureVerifierConstantTime(paramsDeterministc, pk, revealedMsgs, true);
const verifierStatements = new Statements();
verifierStatements.add(verifierStatement1);

const verifierMS = new MetaStatements();
const verifierProofSpec = new ProofSpec(verifierStatements, verifierMS, [], stringToBytes("Awesome Proof - test 001"));

const verifierWitness1 = Witness.bbsPlusSignature(sig, unrevealedMsgs, true);
const verifierWitnesses = new Witnesses();
verifierWitnesses.add(verifierWitness1);

const verifierNonce = stringToBytes('a unique nonce given by verifier');

proof.verify(verifierProofSpec, verifierNonce)

{ verified: true, error: undefined }


In [8]:
proof.hex

0100000000000000009317502974448bd3739187e89f00ef3ad6f8c884497626bf38a10301f9d39fe9bc7dd86079dd1ffe77cfa79ebec80e0c8853e4683ff891893083808a138a0d3707b0c1a7a41b261da1a02038b688d68b035fb155531df633dc1dec8b2c05ba848e616efb9eebcb5f426276a87adc55c50e65f68d5e8ddc607d7f8716a8145848652f0867c9dfa5aa70ae6c98ab9993c5b332899d8f9dd47e661cc3cfd656495799e33b5b6a101d3e12707dd52ff6c863af27c5443ec9ce2138780e861c5d9f4a8ea11c2d67c09d37e5a7ecae19e074072589d09bf8baa987afe35eebab3b730b6225f9dd37caa9309b925cdb4938d082fb08a7c3bedf02b19d3fde9df3d7ee1395734ec475d6c19624bc389d9ec1b15c512cccc2d75b8d09fe01887b50dca8a60b06425bd1eac9fff529e583a7a3787504000000000000002809ea57eb2a9635acae2abb0371f78c25cab23b9428e3062472925e9eddf45bab918edf0e7290970787b50b5b49f76c9219a1e53b8969197d569bf80ca500368b1360d2e572e390fa144c92443cedac98735501ccf41ca2f8f6f0b63ca6d465b56edfc8a5fa6f2616b51aa16d8bdea1cdb6c0744ab0c816722bad7e5633595300000000000000000000


In [14]:
// Generating another proof 
const proof2 = CompositeProof.generate(proofSpec, witnesses, nonce);
proof == proof2  
// false -- different proofs even for the same Verifiable credential
// This results in unlinkability between the different proofs (with regards to verifiers)

false
